In [1]:
import io
from typing import Iterable, Callable
import zipfile
import traceback
from dataclasses import dataclass

import requests


@dataclass
class RawRepositoryFile:
    filename: str
    content: str


class GithubRepositoryDataReader:
    """
    Downloads and parses markdown and code files from a GitHub repository.
    """

    def __init__(self,
                repo_owner: str,
                repo_name: str,
                allowed_extensions: Iterable[str] | None = None,
                filename_filter: Callable[[str], bool] | None = None
        ):
        """
        Initialize the GitHub repository data reader.
        
        Args:
            repo_owner: The owner/organization of the GitHub repository
            repo_name: The name of the GitHub repository
            allowed_extensions: Optional set of file extensions to include
                    (e.g., {"md", "py"}). If not provided, all file types are included
            filename_filter: Optional callable to filter files by their path
        """
        prefix = "https://codeload.github.com"
        self.url = (
            f"{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main"
        )

        if allowed_extensions is not None:
            self.allowed_extensions = {ext.lower() for ext in allowed_extensions}

        if filename_filter is None:
            self.filename_filter = lambda filepath: True
        else:
            self.filename_filter = filename_filter

    def read(self) -> list[RawRepositoryFile]:
        """
        Download and extract files from the GitHub repository.
        
        Returns:
            List of RawRepositoryFile objects for each processed file
            
        Raises:
            Exception: If the repository download fails
        """
        resp = requests.get(self.url)
        if resp.status_code != 200:
            raise Exception(f"Failed to download repository: {resp.status_code}")

        zf = zipfile.ZipFile(io.BytesIO(resp.content))
        repository_data = self._extract_files(zf)
        zf.close()

        return repository_data

    def _extract_files(self, zf: zipfile.ZipFile) -> list[RawRepositoryFile]:
        """
        Extract and process files from the zip archive.
        
        Args:
            zf: ZipFile object containing the repository data

        Returns:
            List of RawRepositoryFile objects for each processed file
        """
        data = []

        for file_info in zf.infolist():
            filepath = self._normalize_filepath(file_info.filename)

            if self._should_skip_file(filepath):
                continue

            try:
                with zf.open(file_info) as f_in:
                    content = f_in.read().decode("utf-8", errors="ignore")
                    if content is not None:
                        content = content.strip()

                    file = RawRepositoryFile(
                        filename=filepath,
                        content=content
                    )
                    data.append(file)

            except Exception as e:
                print(f"Error processing {file_info.filename}: {e}")
                traceback.print_exc()
                continue

        return data

    def _should_skip_file(self, filepath: str) -> bool:
        """
        Determine whether a file should be skipped during processing.
        
        Args:
            filepath: The file path to check
            
        Returns:
            True if the file should be skipped, False otherwise
        """
        filepath = filepath.lower()

        # directory
        if filepath.endswith("/"):
            return True

        # hidden file
        filename = filepath.split("/")[-1]
        if filename.startswith("."):
            return True

        if self.allowed_extensions:
            ext = self._get_extension(filepath)
            if ext not in self.allowed_extensions:
                return True

        if not self.filename_filter(filepath):
            return True

        return False

    def _get_extension(self, filepath: str) -> str:
        """
        Extract the file extension from a filepath.
        
        Args:
            filepath: The file path to extract extension from
            
        Returns:
            The file extension (without dot) or empty string if no extension
        """
        filename = filepath.lower().split("/")[-1]
        if "." in filename:
            return filename.rsplit(".", maxsplit=1)[-1]
        else:
            return ""

    def _normalize_filepath(self, filepath: str) -> str:
        """
        Removes the top-level directory from the file path inside the zip archive.
        'repo-main/path/to/file.py' -> 'path/to/file.py'
        
        Args:
            filepath: The original filepath from the zip archive
            
        Returns:
            The normalized filepath with top-level directory removed
        """
        parts = filepath.split("/", maxsplit=1)
        if len(parts) > 1:
            return parts[1]
        else:
            return parts[0]

In [2]:
def read_github_data():
    repo_owner = 'evidentlyai'
    repo_name = 'docs'
    
    allowed_extensions = {"md", "mdx"}

    reader = GithubRepositoryDataReader(
        repo_owner,
        repo_name,
        allowed_extensions=allowed_extensions,
    )
    
    return reader.read()

In [3]:
!uv add python-frontmatter

Resolved 183 packages in 0.66ms
Audited 177 packages in 2ms


In [4]:
github_data = read_github_data()

In [16]:
print(github_data[3].content)

---
title: 'Introduction'
description: 'Example section for showcasing API endpoints'
---

<Note>
  If you're not looking to build API reference documentation, you can delete
  this section by removing the api-reference folder.
</Note>

## Welcome

There are two ways to build API documentation: [OpenAPI](https://mintlify.com/docs/api-playground/openapi/setup) and [MDX components](https://mintlify.com/docs/api-playground/mdx/configuration). For the starter kit, we are using the following OpenAPI specification.

<Card
  title="Plant Store Endpoints"
  icon="leaf"
  href="https://github.com/mintlify/starter/blob/main/api-reference/openapi.json"
>
  View the OpenAPI specification file
</Card>

## Authentication

All API endpoints are authenticated using Bearer tokens and picked up from the specification file.

```json
"security": [
  {
    "bearerAuth": []
  }
]
```


In [7]:
import frontmatter

def parse_data(data_raw):
    data_parsed = []
    for f in data_raw:
        post = frontmatter.loads(f.content)
        data = post.to_dict()
        data['filename'] = f.filename
        data_parsed.append(data)

    return data_parsed

In [8]:
parsed_data = parse_data(github_data)

In [14]:
print(parsed_data[10]['content'])

You can view or export Reports in multiple formats.

**Pre-requisites**:

* You know how to [generate Reports](/docs/library/report).

## Log to Workspace

You can save the computed Report in Evidently Cloud or your local workspace.

```python
ws.add_run(project.id, my_eval, include_data=False)
```

<Info>
  **Uploading evals**. Check Quickstart examples [for ML](/quickstart_ml) or [for LLM](/quickstart_llm) for a full workflow.
</Info>

## View in Jupyter notebook

You can directly render the visual summary of evaluation results in interactive Python environments like Jupyter notebook or Colab.

After running the Report, simply call the resulting Python object:

```python
my_report
```

This will render the HTML object directly in the notebook cell.

## HTML

You can also save this interactive visual Report as an HTML file to open in a browser:

```python
my_report.save_html(“file.html”)
```

This option is useful for sharing Reports with others or if you're working in a Python enviro

In [10]:
from typing import Any, Dict, Iterable, List


def sliding_window(
        seq: Iterable[Any],
        size: int,
        step: int
    ) -> List[Dict[str, Any]]:
    """
    Create overlapping chunks from a sequence using a sliding window approach.

    Args:
        seq: The input sequence (string or list) to be chunked.
        size (int): The size of each chunk/window.
        step (int): The step size between consecutive windows.

    Returns:
        list: A list of dictionaries, each containing:
            - 'start': The starting position of the chunk in the original sequence
            - 'content': The chunk content

    Raises:
        ValueError: If size or step are not positive integers.

    Example:
        >>> sliding_window("hello world", size=5, step=3)
        [{'start': 0, 'content': 'hello'}, {'start': 3, 'content': 'lo wo'}]
    """
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        batch = seq[i:i+size]
        result.append({'start': i, 'content': batch})
        if i + size > n:
            break

    return result


def chunk_documents(
        documents: Iterable[Dict[str, str]],
        size: int = 2000,
        step: int = 1000,
        content_field_name: str = 'content'
) -> List[Dict[str, str]]:
    """
    Split a collection of documents into smaller chunks using sliding windows.

    Takes documents and breaks their content into overlapping chunks while preserving
    all other document metadata (filename, etc.) in each chunk.

    Args:
        documents: An iterable of document dictionaries. Each document must have a content field.
        size (int, optional): The maximum size of each chunk. Defaults to 2000.
        step (int, optional): The step size between chunks. Defaults to 1000.
        content_field_name (str, optional): The name of the field containing document content.
                                          Defaults to 'content'.

    Returns:
        list: A list of chunk dictionaries. Each chunk contains:
            - All original document fields except the content field
            - 'start': Starting position of the chunk in original content
            - 'content': The chunk content

    Example:
        >>> documents = [{'content': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, size=100, step=50)
        >>> # Or with custom content field:
        >>> documents = [{'text': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, content_field_name='text')
    """
    results = []

    for doc in documents:
        doc_copy = doc.copy()
        doc_content = doc_copy.pop(content_field_name)
        chunks = sliding_window(doc_content, size=size, step=step)
        for chunk in chunks:
            chunk.update(doc_copy)
        results.extend(chunks)

    return results

In [11]:
chunks = chunk_documents(parsed_data)

In [13]:
chunks[29]

{'start': 0,
 'content': 'This page shows the core eval workflow with the Evidently library and links to guides.\n\n## Define and run the eval\n\n<Tip>\n  To log the evaluation results to the Evidently Platform, first connect to [Evidently Cloud](/docs/setup/cloud) or your [local workspace](/docs/setup/self-hosting) and [create a Project](/docs/platform/projects_manage). It\'s optional: you can also run evals locally.\n</Tip>\n\n<Steps>\n  <Step title="Prepare the input data">\n    Get your data in a table like a `pandas.DataFrame`. More on [data requirements](/docs/library/overview#dataset). You can also [load data](/docs/platform/datasets_workflow) from Evidently Platform, like tracing or synthetic datasets.\n  </Step>\n\n  <Step title="Create a Dataset object">\n    Create a Dataset object with `DataDefinition()` that specifies column role and types. You can also use default type detection. [How to set Data Definition](/docs/library/data_definition).\n\n    ```python\n    eval_data 

In [12]:
chunks[30]

{'start': 1000,
 'content': '      data_definition=DataDefinition()\n    )\n    ```\n  </Step>\n\n  <Step title="(Optional) Add descriptors">\n    For text evals, choose and compute row-level `descriptors`. Optionally, add row-level tests to get pass/fail for specific inputs. [How to use Descriptors](/docs/library/descriptors).\n\n    ```python\n    eval_data.add_descriptors(descriptors=[\n        TextLength("Question", alias="Length"),\n        Sentiment("Answer", alias="Sentiment")\n    ])\n    ```\n  </Step>\n\n  <Step title="Configure Report">\n    For dataset-level evals (classification, data drift) or to summarize descriptors, create a `Report` with chosen `metrics`  or `presets`. How to [configure Reports](/docs/library/report).\n\n    ```python\n    report = Report([\n        DataSummaryPreset()\n    ])\n    ```\n  </Step>\n\n  <Step title="(Optional) Add Test conditions">\n    Add dataset-level Pass/Fail conditions, like to check if all texts are in \\< 100 symbols length. How

In [17]:
from minsearch import Index

In [18]:
index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [23]:
search_results = index.search('how do I use llm-as-a-judge for evals')
print(len(search_results))
search_results[0]

10


{'start': 0,
 'content': 'import CloudSignup from \'/snippets/cloud_signup.mdx\';\nimport CreateProject from \'/snippets/create_project.mdx\';\n\nIn this tutorial, we\'ll show how to evaluate text for custom criteria using LLM as the judge, and evaluate the LLM judge itself.\n\n<Info>\n  **This is a local example.** You will run and explore results using the open-source Python library. At the end, we’ll optionally show how to upload results to the Evidently Platform for easy exploration.\n</Info>\n\nWe\'ll explore two ways to use an LLM as a judge:\n\n- **Reference-based**. Compare new responses against a reference. This is useful for regression testing or whenever you have a "ground truth" (approved responses) to compare against.\n- **Open-ended**. Evaluate responses based on custom criteria, which helps evaluate new outputs when there\'s no reference available.\n\nWe will focus on demonstrating **how to create and tune the LLM evaluator**, which you can then apply in different contex

In [20]:
def search(query):
    return index.search(
        query=query,
        num_results=15
    )

In [ ]:
question = 'how do I use llm-as-a-judge for evals'

In [24]:
instructions = """
You're an assistant that helps with the documentation.
Answer the QUESTION based on the CONTEXT from the search engine of our documentation.

Use only the facts from the CONTEXT when answering the QUESTION.

When answering the question, provide the reference to the file with the source.
Use the filename field for that. The repo url is: https://github.com/evidentlyai/docs/
Include code examples when relevant. 
If the question is discussed in multiple documents, cite all of them.

Don't use markdown or any formatting in the output.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

In [30]:
(json.dumps(search_results))

'[{"start": 0, "content": "import CloudSignup from \'/snippets/cloud_signup.mdx\';\\nimport CreateProject from \'/snippets/create_project.mdx\';\\n\\nIn this tutorial, we\'ll show how to evaluate text for custom criteria using LLM as the judge, and evaluate the LLM judge itself.\\n\\n<Info>\\n  **This is a local example.** You will run and explore results using the open-source Python library. At the end, we\\u2019ll optionally show how to upload results to the Evidently Platform for easy exploration.\\n</Info>\\n\\nWe\'ll explore two ways to use an LLM as a judge:\\n\\n- **Reference-based**. Compare new responses against a reference. This is useful for regression testing or whenever you have a \\"ground truth\\" (approved responses) to compare against.\\n- **Open-ended**. Evaluate responses based on custom criteria, which helps evaluate new outputs when there\'s no reference available.\\n\\nWe will focus on demonstrating **how to create and tune the LLM evaluator**, which you can then 

In [ ]:
import json

def build_prompt(question, search_results):
    context = json.dumps(search_results)

    prompt = prompt_template.format(
        question=question,
        context=context
    ).strip()
    
    return prompt

In [31]:
from openai import OpenAI

openai_client = OpenAI()

def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [32]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    return response

In [33]:
result = rag('How can I build an eval report with llm as a judge?')

In [35]:
print(result)

Creating an evaluation report using a large language model (LLM) as a judge can be effectively accomplished through the following steps:

### 1. **Setup and Installation**
Ensure you have the required libraries installed. You will need `Evidently`, which you can install via pip:
```bash
pip install evidently
```

### 2. **Import Necessary Libraries**
Start by importing the relevant modules in your Python script or Jupyter Notebook:
```python
import os
import pandas as pd
from evidently import Dataset, DataDefinition, Report
from evidently.llm.templates import BinaryClassificationPromptTemplate
from evidently.presets import TextEvals
```

### 3. **Set Up OpenAI API Key**
Set your OpenAI API key to enable LLM usage:
```python
os.environ["OPENAI_API_KEY"] = "YOUR_KEY"
```

### 4. **Create Your Evaluation Dataset**
Develop a dataset that includes:
- **Questions**: Inputs for the LLM.
- **Target Responses**: Approved responses you deem accurate.
- **New Responses**: AI-generated or imitated